# Émissions de CO2 pour les véhicules en France d'après l'ADEME

Source : [https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/](https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/)

## Installation de modules et imports

In [1]:
import os
os.system("pip install --quiet requests")

0

In [2]:
import requests
import zipfile
import pandas as pd

## Véhicules

In [3]:
file_url = "https://www.data.gouv.fr/fr/datasets/r/da84abee-6038-43ea-b316-cdaea2514f66"
file_name = "carlab_mars_2014.zip"

In [4]:
response = requests.get(file_url)

In [5]:
response.status_code

200

In [6]:
with open(file_name, 'wb') as f:
    f.write(response.content)

In [7]:
with zipfile.ZipFile(file_name, "r") as zip_obj:
    zip_obj.extractall()

In [8]:
voitures = pd.read_csv("mars-2014-complete.csv", encoding="iso-8859-1", sep=";")

In [9]:
to_drop = voitures.columns[-4:]
voitures = voitures.drop(columns=to_drop)

## Dictionnaire des variables

In [10]:
url_dico = "https://www.data.gouv.fr/fr/datasets/r/e4ed13fe-a8c3-41fb-b9a6-73f73da6dc2b"
file_name_dico = "dictionnaire_variables.xlsx"
response_dico = requests.get(url_dico)

In [11]:
with open(file_name_dico, 'wb') as f:
    f.write(response_dico.content)

In [12]:
var_dico = pd.read_excel(file_name_dico)

In [13]:
var_dico

,nom-colonne,typerubrique,longueur,légende,unité
0,lib_mrq_utac,varchar,12,la marque,NaN
1,lib_mod_doss,varchar,20,le modele du dossier,NaN
2,lib_mod,varchar,20,le modèle commercial,NaN
3,dscom,varchar,91,la désignation commerciale,NaN
4,cnit,varchar,15,le Code National d'Identification du Type (CNIT),NaN
5,tvv,varchar,36,le Type-Variante-Version (TVV) ou le type Mines,NaN
6,cod_cbr,varchar,5,le type de carburant,NaN
7,hybride,varchar,3,une information permettant d’identifier les vé...,NaN
8,puiss_admin_98,varnb,2,la puissance administrative,NaN
9,puiss_max,varnb,11,la puissance maximale (en kW),kW


### Data cleaning

In [57]:
# renommer les colonnes qui nous interessent et ne garder qu'elles
renaming_cols = {
    "lib_mrq": "marque",
    "lib_mod": "modele",
    "dscom": "designation",
    "cod_cbr": "carburant",
    "co2": "co2_g_par_km",
    "puiss_admin_98": "chevaux_fiscaux",
    
}

to_keep = list(renaming_cols.values())
data = voitures.rename(columns=renaming_cols)[to_keep]
# renommer les valeurs dans la colonne carburant
carburants = {'GO': 'Gazole', 'ES': 'Essence', 'EL': 'Electrique'}
data.carburant = data.carburant.map(carburants)
# remplacer les NaN dans les emissions par des 0
data.co2_g_par_km = data.co2_g_par_km.fillna(0)

In [58]:
data.head()

,marque,modele,designation,carburant,co2_g_par_km,chevaux_fiscaux
0,ALFA-ROMEO,159,159 1750 Tbi (200ch),Essence,182.0,12
1,ALFA-ROMEO,159,159 1750 Tbi (200ch),Essence,186.0,12
2,ALFA-ROMEO,159,159 2.0 JTDm (136ch),Gazole,134.0,7
3,ALFA-ROMEO,159,159 2.0 JTDm (136ch),Gazole,134.0,7
4,ALFA-ROMEO,159,159 2.0 JTDm (170ch),Gazole,139.0,9


In [59]:
data.groupby('marque').mean()

,co2_g_par_km,chevaux_fiscaux
marque,,
ALFA-ROMEO,130.901639,7.797814
ASTON MARTIN,324.948718,38.410256
AUDI,151.464183,12.919771
BENTLEY,304.384615,48.153846
BMW,150.462203,14.362851
CADILLAC,237.973684,23.368421
CHEVROLET,179.855263,14.434211
CITROEN,128.759091,6.468182
DACIA,122.344828,5.137931


In [60]:
data[(data.marque == 'PEUGEOT') & (data.modele == '308')]

,marque,modele,designation,carburant,co2_g_par_km,chevaux_fiscaux
40362,PEUGEOT,308,308 CC 1.6 e-Hdi FAP BVM6,Gazole,119.0,6
40363,PEUGEOT,308,308 CC 1.6 THP BVA6 (155ch),Essence,175.0,9
40364,PEUGEOT,308,308 CC 1.6 THP BVM6 (155ch),Essence,160.0,9
40365,PEUGEOT,308,308 CC 1.6 Vti BVM5,Essence,155.0,7
40366,PEUGEOT,308,308 CC 2.0 HDi FAP BVA6 (160ch),Gazole,172.0,10
40367,PEUGEOT,308,308 CC 2.0 HDi FAP BVM6 (160ch),Gazole,144.0,9
40368,PEUGEOT,308,308 SW 1.6 HDi FAP BVM5 (92ch),Gazole,119.0,5
40369,PEUGEOT,308,308 SW 1.6 HDi FAP BVM5 (92ch),Gazole,119.0,5
40370,PEUGEOT,308,308 SW 1.6 VTi BVM5,Essence,155.0,7
40371,PEUGEOT,308,308 SW 1.6L e-HDi (115ch) FAP BVM6,Gazole,120.0,6


In [90]:
def list_makers(data):
    return data.marque.unique().tolist()

In [91]:
list_makers(data)

['ALFA-ROMEO',
 'ASTON MARTIN',
 'AUDI',
 'BENTLEY',
 'BMW',
 'CADILLAC',
 'CHEVROLET',
 'CITROEN',
 'DACIA',
 'FERRARI',
 'FIAT',
 'FORD',
 'HONDA',
 'HYUNDAI',
 'INFINITI',
 'JAGUAR',
 'JEEP',
 'KIA',
 'LADA',
 'LAMBORGHINI',
 'LANCIA',
 'LAND ROVER',
 'LEXUS',
 'LOTUS',
 'MASERATI',
 'MAZDA',
 'MERCEDES',
 'MIA',
 'MINI',
 'MITSUBISHI',
 'NISSAN',
 'OPEL',
 'PEUGEOT',
 'PORSCHE',
 'RENAULT',
 'ROLLS-ROYCE',
 'SEAT',
 'SKODA',
 'SMART',
 'SSANGYONG',
 'SUBARU',
 'SUZUKI',
 'TESLA',
 'TOYOTA',
 'VOLKSWAGEN',
 'VOLVO']

In [118]:
def show_models(data, maker):
    return data[data.marque == maker.upper()][ 'designation'].tolist()


In [119]:
show_models(data, 'peugeot')

['107 3P 1.0e Blue Lion',
 '107 3P 1.0e Blue Lion',
 '107 5P 1.0e 2-tronic Blue Lion BVMP',
 '107 5P 1.0e Blue Lion BVM5',
 '2008 1.2 Vti BVM5',
 '2008 1.2 Vti ETG5',
 '2008 1.4 Hdi FAP BVM5',
 '2008 1.6 e-Hdi FAP BVM5',
 '2008 1.6 e-Hdi FAP BVM6',
 '2008 1.6 e-Hdi FAP ETG6',
 '2008 1.6 Vti BVA',
 '2008 1.6 Vti BVM5',
 '207 CC 1.6 HDi BVM6',
 '207 CC 1.6 HDi BVM6',
 '207 CC 1.6 Vti BVA',
 '207 CC 1.6 Vti BVM5',
 '207+ 3P 1.4 HDi FAP Blue Lion',
 '207+ 5P 1.4 HDi FAP Blue Lion',
 '208 3P 1.0 Vti (68ch) BVM5',
 '208 3P 1.2 e-Vti (82ch) ETG5',
 '208 3P 1.2 Vti (82ch) BVM5',
 '208 3P 1.2 Vti (82ch) BVM5',
 '208 3P 1.4 HDi FAP (68ch) BVM5',
 '208 3P 1.6 e-HDi FAP (115ch) BVM6',
 '208 3P 1.6 e-HDi FAP (92ch) BVM5',
 '208 3P 1.6 e-HDi FAP (92ch) BVM5',
 '208 3P 1.6 e-HDi FAP (92ch) BVM5 BLUE LION',
 '208 3P 1.6 THP (155 ch) BVM6',
 '208 3P 1.6 Vti (120ch) BVA4',
 '208 3P 1.6 Vti (120ch) BVM5',
 '208 3P Gti 1.6 THP (200 ch) BVM6',
 '208 5P 1.0 Vti (68 ch) BVM5',
 '208 5P 1.2 e-Vti (82ch) ETG5'

In [106]:
emission = data.loc[data.designation=="CAPTUR dCi (90ch) EDC eco2", "co2_g_par_km"].tolist()[0]

In [107]:
emission

103.0

In [113]:
def trip_emission(data, designation, distance):
    emission = data.loc[data.designation==designation, "co2_g_par_km"].tolist()[0] * distance / 1000
    print(f"Un trajet de {distance} km avec une {designation} entraîne une émission de {emission} kg de CO2.")

In [120]:
trip_emission(data, '308 CC 1.6 Vti BVM5', 100)

Un trajet de 100 km avec une 308 CC 1.6 Vti BVM5 entraîne une émission de 15.5 kg de CO2.


## Emissions moyennes par type de carburant 

In [123]:
data.groupby('carburant')[['co2_g_par_km']].mean()

,co2_g_par_km
carburant,
Electrique,0.000000
Essence,203.757593
Gazole,202.151853


## Emissions moyennes par marque et par type de carburant 

In [131]:
carb_marque = data.groupby(['carburant', 'marque'])[['co2_g_par_km']].mean().sort_values('co2_g_par_km', ascending=True)
carb_marque

co2_g_par_km
carburant  marque                    
Electrique BMW               0.000000
           VOLKSWAGEN        0.000000
           TESLA             0.000000
           RENAULT           0.000000
           PEUGEOT           0.000000
...                               ...
Essence    BENTLEY         304.384615
           FERRARI         305.913043
           ASTON MARTIN    324.948718
           ROLLS-ROYCE     338.636364
           LAMBORGHINI     341.666667

[91 rows x 1 columns]

In [132]:
carb_marque.loc['Essence']

,co2_g_par_km
marque,
SMART,112.050000
SEAT,124.062500
FIAT,127.369318
DACIA,128.181818
SKODA,131.206349
RENAULT,132.307692
TOYOTA,132.565217
LANCIA,132.838710
MITSUBISHI,133.875000


In [133]:
carb_marque.loc['Gazole']

,co2_g_par_km
marque,
SMART,86.500000
MINI,111.122807
DACIA,112.285714
ALFA-ROMEO,116.402985
SKODA,118.558824
SEAT,118.733333
MAZDA,126.150000
SUZUKI,126.400000
CITROEN,127.455172
